In [ ]:
a = [1, 2]
b = [4]
a += b
a

We can control the number of concurrent requests manually by using a semaphore to only do HTTP gets from 100 greenlets at a time. A semaphore works by making sure that only a certain number of coroutines can enter the context block at a time. As a result, we can launch all the greenlets that we need in order to fetch the URLs right away, but only 100 of them will be able to make HTTP calls at a time. Semaphores are one type of locking mechanism used a lot in various parallel code flows. By restricting the progression of your code based on various rules, locks can help you make sure that the various components of your program don’t interfere with each other.


In [ ]:
import time

def parallelizable_process():
    """though unclear how the cursor works here"""
    print("pp")

def queue_operation():
    print("queuing")

def asych_save_operation():
    """"""
    print("ready saving")
    time.sleep(5)
    print("saving done")

In [ ]:
def cursor(limit=4):
    i = 0
    while i < limit:
        print("cursor %s" % i)
        yield i
        i+= 1

In [ ]:
import asyncio

async def do_thing(i):
    print("do thing %s ..." % (i))
    await asyncio.sleep(1.0)
    return i

async def controller():
    i = 1
    result = await do_thing(i)
    print("controller %s " % (result))

# if asyncio._check_closed() and asyncio.is_running():
#     loop.
    
loop = asyncio.get_event_loop()
loop.run_until_complete(controller())
loop.close()


In [ ]:
from collections import deque

queue = deque()
size = 10

async def client_stage_batch(batch):
    """Handles sending the result to the server.
    Once the send is executed (not the successul completion)
    it will complete the future
    """
    future = asyncio.Future()
# ???????
    yield asyncio.ensure_future(slow_operation(future))


async def queue_side_handler(resultList):
    [queue.append(r) for r in resultList ]
    if len(queue >= size):
        result = await client_stage_batch(queue)
        # clear queue
        return result
    return True

# loop.run_until_complete(future)
# print(future.result())
# loop.close()


In [ ]:
async def processor_side_enque(resultList):
    """Sends the list of results to the queue from the processor
    The queue side returns a future which resolves after the 
    result list has been enqueued or the request has been sent to the 
    server (if the batch size has been reached)
    """
    result = await queue_side_handler(resultList)
    return result
    

# start an ipcontroller and cluster first

%%bash 
ipcluster start -n 2

In [ ]:

%cd twitteranalysis
import ipyparallel as ipp
# Connect to the cluster
rc = ipp.Client()

In [ ]:
%magic

# Example

In [ ]:
with rc[:].sync_imports():
    import numpy

In [ ]:
%px a = numpy.random.rand(2,2)


In [ ]:
%px numpy.linalg.eigvals(a)


In [ ]:
%%px --noblock --targets 1
print ('hi')


In [ ]:
%pxresult

In [ ]:
%%px --noblock --targets 0
print ('ho')


In [ ]:
%pxresult

In [ ]:
%%px --noblock
%cd twitteranalysis
from Servers import DatabaseServerGrumble as DSG 
j = DSG.main()

In [ ]:
%cd twitteranalysis

In [ ]:
%pxresult

In [ ]:
%%px --noblock --targets 1
%cd twitteranalysis

from Executables import process_user_descriptions_into_words as Runner
Runner.main()

In [ ]:
%pxresult

In [ ]:
import ipyparallel as ipp
rc = ipp.Client()



# lview = rc.load_balanced_view() 
# lview.block = True

# @lview.parallel()
# def start_db():
#     DSG.main()

# @lview.parallel()
# def run():
#     Runner.main()

# %%px --targets 1 

In [ ]:
rc.ids

In [ ]:

e1 = rc[0]
e2 = rc[1]
e1.apply(DSG.main())


In [ ]:
start_db()
print ('j')
run()

In [ ]:
%%px --noblock
print ('hi')



In [ ]:
%pxresult

# Based on quick and easy section

In [ ]:
#Run db in separate notebook



In [ ]:
%cd twitteranalysis
from Servers.DatabaseServerGrumble import main

In [ ]:
main()

In [ ]:
test()

In [ ]:
# Then we load the data in and look to see if all the timestamps
# for any of the operations cluster in ways we don't want them to

In [ ]:
🐈  TwitterDataAnalysis $python Servers/DatabaseServerGrumble.py

Or, you know, we could just run them in separate ipython notebooks
since those are handled by separate kernels